In [1]:
!pip install nba_api
import json
import pandas
import requests
import nba_api
import pandas as pd

from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
response = shotchartdetail.ShotChartDetail(
	team_id=0,
	player_id=201935,
	season_nullable='2018-19',
	season_type_all_star='Regular Season'
)

content = json.loads(response.get_json())

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [4]:
all_players = players.get_players()
all_players_id = [p['id'] for p in all_players]

In [5]:
len(all_players_id)

4831

In [6]:
active_players = []
for p in all_players:
    if p['is_active'] == True:
        active_players.append(p['id'])
        
print(len(active_players))

582


In [7]:
active_players[:3]

[1630173, 203500, 1628389]

In [8]:
season_years = [i for i in range(1990, 2022)]

In [9]:
import requests
import json


In [10]:
def get_season_no(season_year):
    assert 1990 <= season_year <= 2022
    return str(season_year) + '-' + str(season_year+1)[-2:]

In [11]:
get_season_no(2000)

'2000-01'

In [12]:
def get_data_player_year(player_id, season_year):
    url_base = 'https://stats.nba.com/stats/shotchartdetail'
    print("player_id",player_id)
    season = get_season_no(season_year)
    print("season",season)
    print('here1')
    headers = {
            'Host': 'stats.nba.com',
            'Connection': 'keep-alive',
            'Accept': 'application/json, text/plain, */*',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
            'Referer': 'https://stats.nba.com/',
            "x-nba-stats-origin": "stats",
            "x-nba-stats-token": "true",
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9',
        }
    print('here2')
    parameters = {
        'ContextMeasure': 'FGA',
        'LastNGames': 0,
        'LeagueID': '00',
        'Month': 0,
        'OpponentTeamID': 0,
        'Period': 0,
        'PlayerID': player_id,
        'SeasonType': 'Regular Season',
        'TeamID': 0,
        'VsDivision': '',
        'VsConference': '',
        'SeasonSegment': '',
        'Season': season,
        'RookieYear': '',
        'PlayerPosition': '',
        'Outcome': '',
        'Location': '',
        'GameSegment': '',
        'GameId': '',
        'DateTo': '',
        'DateFrom': ''
    }
    print('here3')
    response = requests.get(url_base, params=parameters, headers=headers)
    print('here4')
    content = json.loads(response.content)
    print('here5')
    # transform contents into dataframe
    results = content['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    df = pd.DataFrame(rows)
    df.columns = headers
    return df
    # print('here')
    # print(list(df.iterrows()))
    # print("bye")
    # return list(df.iterrows())

In [13]:
def get_player_data(player_id):
    url_base = 'https://stats.nba.com/stats/commonplayerinfo?LeagueID=&PlayerID='
    print("player_id",player_id)
    print('here1')
    headers = {
            'Host': 'stats.nba.com',
            'Connection': 'keep-alive',
            'Accept': 'application/json, text/plain, */*',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
            'Referer': 'https://stats.nba.com/',
            "x-nba-stats-origin": "stats",
            "x-nba-stats-token": "true",
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9',
        }
    print('here2')
    response = requests.get(url_base+str(player_id), headers=headers)
    print('here4')
    content = json.loads(response.content)
    print('here5')
    # transform contents into dataframe
    results = content['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    df = pd.DataFrame(rows)
    df.columns = headers
    return df
    # print('here')
    # print(list(df.iterrows()))
    # print("bye")
    # return list(df.iterrows())

In [ ]:
trial = get_player_data(76001)
print(trial['FROM_YEAR'][0])
print(trial['TO_YEAR'][0])

player_id 76001
here1
here2


In [ ]:
tttt = get_data_player_year(203500, 2020)

In [ ]:
df = {}

import time
print("helo")
for i in range(3500,4000):
    player_id = all_players_id[i]
    print("player is", player_id)
    player_info = get_player_data(player_id)
    from_year = player_info['FROM_YEAR'][0]
    to_year = player_info['TO_YEAR'][0]
    
    if to_year >= 1990:
        for season_year in range(max(from_year, 1990), min(to_year, 2021)+1):
            print(season_year)
            try:
                time.sleep(1)
                dft = get_data_player_year(player_id, season_year)
                dft.to_csv(str(player_id)+'-'+str(season_year)+'.csv', index=False)
                print('dft done')
                if season_year not in df:
                  df[season_year] = dft
                else:
                  print('check')
                  df[season_year] = pd.concat([df[season_year], dft], axis=0)
                print('success')
            except:
                print('unsuccessful')
                pass


# write to csv file
# df.to_csv(str(parameters['PlayerID'])+'.csv', index=False)

In [ ]:
print(df['2020-21'])
df['2020-21'].to_csv('nba.csv', encoding='utf-8', index=False)

In [ ]:
df.SHOT_ZONE_AREA.unique()

array(['Center(C)', 'Right Side(R)', 'Left Side(L)',
       'Left Side Center(LC)'], dtype=object)

In [ ]:
def calcPoints(x, y):
    if y < 115:
        return 2 if abs(x) < 220 else 3
    else:
        return 2 if x**2 + y**2 < 237.5**2 else 3
        

In [ ]:
dft = df[:10]

In [ ]:
for index, rows in dft.iterrows():
    print(rows['LOC_X'])

11
55
40
-29
15
-12
-12
0
24
-29


In [ ]:
points = [calcPoints(rows['LOC_X'], rows['LOC_Y']) for index, rows in df.iterrows()]

In [ ]:
len(points)

228

In [ ]:
len(df)

228

In [ ]:
df['attempted_points'] = points

In [ ]:
df

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,attempted_points
0,Shot Chart Detail,0022000005,92,1630173,Precious Achiuwa,1610612748,Miami Heat,1,3,40,...,Less Than 8 ft.,1,11,16,1,1,20201225,MIA,NOP,2
1,Shot Chart Detail,0022000005,226,1630173,Precious Achiuwa,1610612748,Miami Heat,2,6,4,...,Less Than 8 ft.,5,55,16,1,0,20201225,MIA,NOP,2
2,Shot Chart Detail,0022000005,393,1630173,Precious Achiuwa,1610612748,Miami Heat,3,6,1,...,Less Than 8 ft.,4,40,-9,1,0,20201225,MIA,NOP,2
3,Shot Chart Detail,0022000005,402,1630173,Precious Achiuwa,1610612748,Miami Heat,3,5,32,...,Less Than 8 ft.,3,-29,12,1,1,20201225,MIA,NOP,2
4,Shot Chart Detail,0022000005,412,1630173,Precious Achiuwa,1610612748,Miami Heat,3,4,59,...,Less Than 8 ft.,2,15,17,1,1,20201225,MIA,NOP,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Shot Chart Detail,0022001069,448,1630173,Precious Achiuwa,1610612748,Miami Heat,3,1,47,...,Less Than 8 ft.,5,-43,27,1,1,20210516,DET,MIA,2
224,Shot Chart Detail,0022001069,550,1630173,Precious Achiuwa,1610612748,Miami Heat,4,4,7,...,Less Than 8 ft.,2,-20,5,1,1,20210516,DET,MIA,2
225,Shot Chart Detail,0022001069,570,1630173,Precious Achiuwa,1610612748,Miami Heat,4,1,59,...,Less Than 8 ft.,1,-13,6,1,0,20210516,DET,MIA,2
226,Shot Chart Detail,0022001069,572,1630173,Precious Achiuwa,1610612748,Miami Heat,4,1,55,...,Less Than 8 ft.,0,0,0,1,1,20210516,DET,MIA,2


In [ ]:
def accuracy(df):
    return sum(df['SHOT_MADE_FLAG'])/sum(df['SHOT_ATTEMPTED_FLAG'])

In [ ]:
two_pt_accuracy = accuracy(df[df['attempted_points'] == 2])

In [ ]:
two_pt_accuracy

0.48495186405634166

In [ ]:
three_pt_accuracy = accuracy(df[df['attempted_points'] == 3])

In [ ]:
three_pt_accuracy

0.36705829191143247

In [ ]:
overall_accuracy = accuracy(df)

In [ ]:
overall_accuracy

0.4587348640908406

In [ ]:
len(df[df['attempted_points'] == 2])

154770

In [ ]:
len(df[df['attempted_points'] == 3])

44260